In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline
from sklearn.model_selection import train_test_split
from datetime import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
import nltk
from sklearn.model_selection import train_test_split


In [2]:
tf.__version__

'1.4.0'

In [3]:
# TODO's
## RNN implementation
## maybe add regulariazation for every layer

In [4]:
data = pd.read_csv('Tweets.csv')
data
data = data.loc[:,['text','airline_sentiment']]
data.dropna(inplace=True)
data = data.values.tolist()
print(len(data))

14640


In [5]:
import os
#tranform sentences to words with Tweet tokenizer
def get_tokenized_sentences_and_labels(data):
    sentences = []
    labels = []
    for i in data:
        #sentences.append(nltk.tokenize.word_tokenize(i[0]))
        sentences.append(nltk.tokenize.TweetTokenizer().tokenize(i[0]))
        labels.append(i[1])
    return sentences,labels

#creates the dataset and returns the vocabulary and x,y
def create_dataset_for_sentence_classification(data,n_vocab=None):

    sentences,labels = get_tokenized_sentences_and_labels(data)
    print('creating vocabulary')
    vocab, max_sentence = create_vocab(sentences, n_vocab)
    print('vocabulary lenght: ' + str(len(vocab)))
    x, y = create_training_set(sentences, labels,vocab,max_sentence)
    return vocab, x, y

#creates the dataset and returns the vocabulary and x,y
def create_dataset_for_sentence_classification_with_word2vec(data,n_vocab=None):

    sentences,labels = get_tokenized_sentences_and_labels(data)
    print('creating vocabulary')
    vocab, max_sentence,embeddings,words_to_embeddings = create_vocab_word2vec(sentences, n_vocab)
    print('vocabulary lenght: ' + str(len(vocab)))
    x, y = create_training_set(sentences, labels,vocab,max_sentence,padding='left')
    return vocab, x, y ,max_sentence, embeddings,words_to_embeddings

#creates the vocabulary and returns it alongside with the max length of a sentence for padding.
#Assigns an index for every word. If n_vocab==None then uses all words in dataset
def create_vocab(sentences,n_vocab=None):
    vocab = dict()
    vocab['UNK'] = 0  #for unkonwn words (this might be occured in the testing time or if i define n_vocab)
    vocab['PAD'] = 1  #for padding
    index = 2
    max_sentence = 0
    for sentence in sentences:
        counter = 0 
        for word in sentence:
            counter+=1
            word = word.lower()
            if word not in vocab:
                vocab[word] = index
                index +=1
                if index == n_vocab:
                    return vocab,max_sentence
        if counter > max_sentence:
            max_sentence = counter
                
    return vocab,max_sentence

def create_vocab_word2vec(sentences, n_vocab=None):
    # create vocabulary
    embeddings = []
    words_to_embeddings = []
    v = {}
    aa = 0
    v['PAD'] = aa
    embeddings.append(np.zeros(shape=(300)))
    words_to_embeddings.append('PAD')
    word2vec,_,_ = get_word2vec()
    exists_in_word2vec = 0
    do_not_exist_in_word2vec = 0
    max_sentence = 0
    for words in sentences:
        counter = 0 
        # print(words)
        for i in words:
            if 'http' not in i and not i.startswith('@') and i != 'RT':
                counter+=1
                if i.lower() not in v:
                    if ((n_vocab is not None) and (aa > (n_vocab - 1))):
                            # v = OrderedDict(sorted(v.items(), key=itemgetter(1)))
                            return v,max_sentence, np.array(embeddings)
                    else:
                        aa += 1
                        v[i.lower()] = aa
                        if i.lower() in word2vec:
                            embeddings.append(word2vec[i.lower()])
                            words_to_embeddings.append(i.lower())
                            exists_in_word2vec+=1
                        else:
                            embeddings.append(np.random.randn(300))
                            words_to_embeddings.append(i.lower())
                            do_not_exist_in_word2vec+=1
            if counter > max_sentence:
                max_sentence = counter
    print('{} exist.{} not exist'.format(exists_in_word2vec,do_not_exist_in_word2vec))
    return v,max_sentence, np.array(embeddings),words_to_embeddings


def create_training_set(sentences,labels,vocabulary,max_sentence,padding='right'):
    x = []
    y = []
    for j,sentence in enumerate(sentences):
        sentence = [i.lower() for i in sentence]
        sentence_data = []
        for word in sentence:
            #print(word)
            if (word in vocabulary):
                sentence_data.append(vocabulary[word])
                
        x.append(sentence_data)
        y.append(label_to_one_hot(labels[j]))
        
    padded_x = []
    for sentence in x:
        if(padding=='right'):
            sentence = sentence + ([0] * (max_sentence - len(sentence) + 1))
        elif(padding=='left'):
            sentence = ([0] * (max_sentence - len(sentence) + 1)) + sentence 
        padded_x.append(sentence)   
    
    return (np.array(padded_x),np.squeeze(np.array(y)))
    #squeeze to remove diamnsion equals to 1

def label_to_one_hot(label):
    a = np.zeros((1,3))
    if label=='positive':    
        a[0,0] =1
    elif label=='negative':
        a[0,2] =1
    else:
        a[0,1] =1
    return a 

def get_word2vec():
    word2vec_dict = {}
    words = []
    embeddings = []
    
    name = './word_vectors/wiki.simple.vec'
    #name = 'D://Downloads/wiki.en.vec'
    with open(name, encoding="utf8") as file:
        for i, line in enumerate(file):
            word = line.split(' ')[0]
            words.append(word.lower())
            embedding = line.split(' ')[1:301]
            embeddings.append(embedding)
            word2vec_dict[word.lower()] = np.asarray(embedding, dtype=float)
    return word2vec_dict,words,embeddings


In [6]:
vocab,x,y,max_sentence,embeddings,words_to_embeddings= create_dataset_for_sentence_classification_with_word2vec(data)
#vocab,x,y= create_dataset_for_sentence_classification(data)

creating vocabulary
8078 exist.6742 not exist
vocabulary lenght: 14821


In [7]:
max_sentence

41

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=41)

In [9]:
def get_batches(X,y, batch_size,shuffle=True):
    """
    Generates a batch generator for a dataset.
    """
    size = X.shape[0]
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(size))
        X = X[shuffle_indices]
        y = y[shuffle_indices]
        
    number_of_batches = int(np.ceil(size/batch_size))
    for i in range(number_of_batches-1):
        index = i*batch_size
        yield X[index:index+batch_size],y[index:index+batch_size]
    yield X[(number_of_batches-1)*batch_size:],y[(number_of_batches-1)*batch_size:]

In [30]:

class DNN():
    
    number = 0
    
    def __init__(self,X_train,y_train,layers,as_placeholders=True,dtypes=[tf.float32,tf.float32]):
        self.X_train = X_train
        self.y_train = y_train
        if as_placeholders:
            self.X = tf.placeholder(dtype=dtypes[0],shape=(None,)+X_train.shape[1:])
            self.y = tf.placeholder(dtype=dtypes[1],shape=(None,)+y_train.shape[1:])
        else:
            self.X = X_train
            self.y = y_train
        self.layers = layers
        self.activations = [self.X]
        self.layers = layers
        DNN.number +=1
            
    def forward(self):       
        for i,layer in enumerate(self.layers):
            layer.set_input(self.activations[i])
            self.activations.append(layer.forward())
        return self.activations
    
    def cost(self):
        #return tf.reduce_sum(tf.square(self.activations[-1]-self.y))
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.activations[-1],labels=self.y))
    
    def squared_cost(self):
        return tf.reduce_mean(tf.squared_difference(self.activations[-1], self.y))
    
class DenseLayer():
    
    number = 0
    
    def __init__(self,units,activation_function=tf.nn.relu,batch_norm=False,keep_prob=1.0):
        self.units = units
        self.keep_prob = keep_prob
        self.activation_function = activation_function
        self.batch_norm = batch_norm
        self.variable_scope_name = 'Dense-'+str(DenseLayer.number)
        DenseLayer.number+=1
        
    def set_input(self,x):
        self.x = x
        if(len(x.shape)==4):
            shape = self.x.get_shape().as_list()        
            dim = np.prod(shape[1:])
            self.x = tf.reshape(tensor=self.x,shape=[-1,dim])


        with tf.variable_scope(self.variable_scope_name):  
            self.init_W((self.x.get_shape().as_list()[1],self.units))
            self.init_b(self.units)
            if(self.batch_norm):
                self.epsilon = 1e-3
                self.scale = tf.get_variable('scale', initializer=tf.ones(shape=[self.units]))
                self.beta =  tf.get_variable('beta', initializer=tf.zeros(shape=[self.units]))

        
    def init_W(self,shape):
        #another way to do this with get variable
        #self.w= tf.Variable(tf.multiply(tf.random_normal(shape),0.01),dtype=tf.float32)
        self.w=tf.get_variable('weight',shape=shape,initializer=tf.contrib.layers.xavier_initializer())
        tf.summary.histogram('weight',self.w)
    
    def init_b(self,shape):
        #self.b = tf.Variable(tf.fill([shape],0.1),dtype=tf.float32)
        init = tf.constant(0.1,shape=[shape])
        self.b = tf.get_variable('bias', initializer=init)
        tf.summary.histogram('bias',self.b)
        
    def forward(self):
        if self.x == None:
            print('no input defined')
        else:
            #batch norm not working properly
            #or maybe it is working properly but it needs bigger batch size. like 64. This makes sense because you make estimates of
            #mean and variance for every z calculation. This means that the bigger the batch size the more accurate the estimate
            if(self.batch_norm):
                self.z = tf.matmul(self.x,self.w)
                self.batch_mean, self.batch_var = tf.nn.moments(self.z,[0])
                self.z  = tf.nn.batch_normalization(self.z,self.batch_mean,self.batch_var,self.beta,self.scale,self.epsilon)
            else:
                self.z = tf.nn.xw_plus_b(self.x,self.w,self.b)
            if self.activation_function == None:
                self.activation = self.z
            else:
                self.activation  = self.activation_function(self.z)
            self.activation = tf.nn.dropout(self.activation,self.keep_prob)
            tf.summary.histogram('activations',self.activation)
            return self.activation
        
class ConvLayer():

    number = 0
    def __init__(self,kernel_size,number_of_kernels,padding='SAME',activation_function=tf.nn.relu,batch_norm = False,keep_prob=1.0):
        self.kernel_size = kernel_size
        self.number_of_kernels = number_of_kernels
        self.padding = padding
        self.activation_function = activation_function
        self.keep_prob = keep_prob
        self.batch_norm = batch_norm
        self.variable_scope_name = 'Conv-'+str(ConvLayer.number)
        ConvLayer.number+=1
        
    def set_input(self,x):
        with tf.variable_scope(self.variable_scope_name):         
            self.x = x
            if(isinstance(self.kernel_size,tuple)):
                self.init_Kernel(self.kernel_size+(x.get_shape().as_list()[-1],self.number_of_kernels))
            else:
                self.init_Kernel((self.kernel_size,self.kernel_size,x.get_shape().as_list()[-1],self.number_of_kernels)) 
            self.init_b(self.number_of_kernels)
            if(self.batch_norm):
                self.epsilon = 1e-3
                self.scale = tf.get_variable('scale', initializer=tf.ones(shape=[self.number_of_kernels]))
                self.beta =  tf.get_variable('beta', initializer=tf.zeros(shape=[self.number_of_kernels]))
        
    def init_Kernel(self,shape):
        self.kernel=tf.get_variable('kernel',shape=shape,initializer=tf.contrib.layers.xavier_initializer())
        tf.summary.histogram('kernel',self.kernel)
        
    def init_b(self,shape):
        self.b = tf.get_variable('bias',shape=[shape],initializer=tf.constant_initializer(0.1))
        tf.summary.histogram('bias',self.b)
        
    def forward(self):
        if self.x == None:
            print('no input defined')
        else:
            if self.batch_norm:
                
                self.z = tf.nn.conv2d(self.x , self.kernel, [1, 1, 1, 1], padding=self.padding)
                self.batch_mean, self.batch_var = tf.nn.moments(self.z,[0,1,2])
                self.z  = tf.nn.batch_normalization(self.z,self.batch_mean,self.batch_var,self.beta,self.scale,self.epsilon)
                
            else:
                self.z = tf.nn.conv2d(self.x , self.kernel, [1, 1, 1, 1], padding=self.padding)
                self.z = tf.nn.bias_add(self.z, self.b)  
                    
            if self.activation_function == None:
                self.activation = self.z
            else:
                self.activation  = self.activation_function(self.z)
            #dropout
            self.activation = tf.nn.dropout(self.activation,self.keep_prob)
            tf.summary.histogram('activations',self.activation)
        return self.activation
        
class PoolingLayer():
    
    number = 0
    
    def __init__(self,kernel_size,stride,padding='SAME',pooling='MAX'):
        self.kernel_size = kernel_size
        self.stride = stride
        self.pooling = pooling
        self.padding = padding
        self.variable_scope_name = 'Pool-'+str(PoolingLayer.number)
        PoolingLayer.number+=1
        
    def set_input(self,x):
        with tf.variable_scope(self.variable_scope_name):         
            self.x = x
        
    def forward(self):
        if self.x == None:
            print('no input defined')
        else:
            if(isinstance(self.kernel_size,tuple)):
                size = list(self.kernel_size)
            else:
                size = [self.kernel_size,self.kernel_size]   
            if (self.pooling == 'MAX'):
                self.activation = tf.nn.max_pool(self.x,[1]+size+[1],[1,self.stride,self.stride,1],padding=self.padding)
            elif (self.pooling == 'AVG'):
                self.activation = tf.nn.avg_pool(self.x,[1]+size+[1],[1,self.stride,self.stride,1],padding=self.padding)
        return self.activation
    
class EmbeddingLayer():
    
    number = 0
    
    def __init__(self,vocabulary_size,embedding_diamension,pretrained_word2vec=True,as_sequences=None):
        self.embedding_diamension = embedding_diamension
        self.vocabulary_size = vocabulary_size
        self.pretrained_word2vec = pretrained_word2vec
        self.variable_scope_name = 'Embedding-'+str(PoolingLayer.number)
        self.init_Embeddings((self.vocabulary_size,self.embedding_diamension))
        self.as_sequences = as_sequences
        EmbeddingLayer.number+=1
        
    def set_input(self,x):
        with tf.variable_scope(self.variable_scope_name):         
            self.x = x
            if self.as_sequences:
                self.sequence_length = x.shape[1]
        
    def init_Embeddings(self,shape):
        with tf.variable_scope(self.variable_scope_name):  
            if(self.pretrained_word2vec):             
                self.W = tf.Variable(tf.constant(0.0, shape=[self.vocabulary_size, self.embedding_diamension]),trainable=True, name="embedding_weights")
                self.embedding_placeholder = tf.placeholder(tf.float32, [self.vocabulary_size, self.embedding_diamension])
                self.embedding_init = self.W.assign(self.embedding_placeholder)
            else:
                self.W = tf.get_variable("embedding_weights", shape=[self.vocabulary_size, self.embedding_diamension],initializer=tf.contrib.layers.xavier_initializer())
    def forward(self):
        # TODO: implement as sequences to provide data for lstm
        if self.x == None:
            print('no input defined')
        else:
            #this is called activation to follow the pattern of other layers
            self.activation = tf.nn.embedding_lookup(self.W, self.x)
            if self.as_sequences == 'static':
                self.activation = tf.transpose(self.activation, [1,0,2])
                self.activation = tf.reshape(self.activation , [-1,self.embedding_diamension])
                self.activation  = tf.split(self.activation ,self.sequence_length,0)
            elif self.as_sequences == 'dynamic':
                pass
            else:
                self.activation = tf.expand_dims(self.activation, -1)        #need 4 diamensions to apply convolution
            return self.activation
class RnnLayer():
    number = 0
    def __init__(self,units,activation_function = None,cell_type = 'LSTM',keep_prob=1.0,rnn_type='static'):
        self.units = units
        self.activation_function = activation_function
        self.keep_prob = keep_prob
        self.cell_type = cell_type
        self.rnn_type = rnn_type
        self.variable_scope_name = 'Rnn-'+str(RnnLayer.number)
        RnnLayer.number+=1
        
    def set_input(self,x):
        with tf.variable_scope(self.variable_scope_name):         
            self.x = x
        
    def forward(self):
        if self.x == None:
            print('no input defined')
        else:
            if self.cell_type == 'LSTM':
                cell = tf.nn.rnn_cell.LSTMCell(self.units,activation=self.activation_function)
                cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=self.keep_prob)
            elif self.cell_type == 'GRU':
                cell = tf.nn.rnn_cell.GRUCell(self.units,activation=self.activation_function)
                cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=self.keep_prob)
                
            #need to fix this
            #self.activation = tf.nn.dynamic_rnn(cell,self.x,sequence_length=self.sequence_length)
            if (self.rnn_type == 'static'):
                self.output, self.states = tf.nn.static_rnn(cell,self.x,dtype=tf.float32)
                self.activation = self.output[-1]
            elif (self.rnn_type == 'dynamic'):
                self.output, self.states = tf.nn.dynamic_rnn(cell,self.x,dtype=tf.float32)
                self.activation = self.output[:,-1,:]
        return self.activation
    

In [42]:
#text classification with rnn
tf.reset_default_graph()
DenseLayer.number = 0
ConvLayer.number = 0
PoolingLayer.number = 0
EmbeddingLayer.number = 0
RnnLayer.number = 0

# X = tf.placeholder(dtype=tf.int32,shape=(None,)+X_train.shape[1:])
# y = tf.placeholder(dtype=tf.float32,shape=(None,)+y_train.shape[1:])

epochs = 50
batch_size = 32

dnn = DNN(X_train,y_train,[EmbeddingLayer(len(vocab),300,pretrained_word2vec=True,as_sequences='static'),\
                               RnnLayer(128,activation_function=tf.nn.relu),DenseLayer(3,activation_function=None)],dtypes=[tf.int32,tf.float32])
output= dnn.forward()[-1]# last activation of th

#- should be the sequence length for the paricular traininng input
loss = dnn.cost()
tf.summary.scalar('loss', loss)
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(dnn.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
#to save model
saver = tf.train.Saver()

with tf.Session() as sess:
    #initialize variables
    sess.run(tf.global_variables_initializer())
    
    #write training data to file in order to visualize it with tensorboard
    log_name = datetime.now().strftime("%Y%m%d-%H%M%S")
    #train_writer = tf.summary.FileWriter('summaries/train/logs'+log_name,sess.graph)
    test_writer = tf.summary.FileWriter('summaries/test/logs'+log_name,sess.graph)
    
    #training 
    for i in tqdm(range(epochs)):
        #mini batch training with my super generator function
        for c,batch in enumerate(get_batches(X_train,y_train,batch_size)):
            batch_x = batch[0]
            batch_y = batch[1]                            
            feed_dict={dnn.X : batch_x,dnn.y : batch_y}
            cost,_,acc= sess.run([loss,train,accuracy],feed_dict=feed_dict)
            #print("cost in "+str(c)+' batch : '+str(cost))
            #print("accuracy in "+str(c)+' batch : '+str(acc))
        
        #test loggingfor every epoch
        feed_dict={dnn.X:X_test, dnn.y:y_test}
        summary,test_acc,test_cost = sess.run([merged,accuracy,loss],feed_dict=feed_dict)
        test_writer.add_summary(summary, i)
        
#         if i%10==0:
#             print('epoch:'+str(i))
            #print("cost after "+str(i)+' epoch : '+str(train_cost))
            #print("accuracy after "+str(i)+' epoch : '+str(train_acc))
           
    #save model (can save it if its better that the previous one etc)
    saver.save(sess,'models/mymodel.ckpt')
        
    feed_dict={dnn.X : X_test,dnn.y : y_test}
    cost,test_acc = sess.run([loss,accuracy],feed_dict=feed_dict)
    print("test accuracy "+str(test_acc))

  2%|█▋                                                                                 | 1/50 [00:14<11:32, 14.13s/it]

KeyboardInterrupt: 

In [49]:
#text classification with convolution
tf.reset_default_graph()
DenseLayer.number = 0
ConvLayer.number = 0
PoolingLayer.number = 0
EmbeddingLayer.number = 0
RnnLayer.number = 0

# X = tf.placeholder(dtype=tf.int32,shape=(None,)+X_train.shape[1:])
# y = tf.placeholder(dtype=tf.float32,shape=(None,)+y_train.shape[1:])
epochs = 50
batch_size = 128

dnn = DNN(X_train,y_train,[EmbeddingLayer(len(vocab),300,pretrained_word2vec=True)],dtypes=[tf.int32,tf.float32])
embedding_layer = dnn.forward()[-1]

dnn_reduce_dimensions = DNN(embedding_layer,dnn.y,[ConvLayer((1,201),16,padding='VALID')],as_placeholders=False)
reduce_dimensions = dnn_reduce_dimensions.forward()[-1]

dnn2 = DNN(reduce_dimensions,dnn.y,[ConvLayer((3,100),32,padding='VALID'),PoolingLayer((40,1),1,'VALID')],as_placeholders=False)
dnn3 = DNN(reduce_dimensions,dnn.y,[ConvLayer((5,100),32,padding='VALID'),PoolingLayer((38,1),1,padding='VALID')],as_placeholders=False)
f2=dnn2.forward()
f3 = dnn3.forward()
#mean of two tensors
mean = tf.divide(tf.add(f2[-1],f3[-1]),2)
#merge two tesnors
merge = tf.concat([f2[-1],f3[-1]],axis=-1)
#this flattens the previous input automatically
dnn4 = DNN(merge,dnn.y,[DenseLayer(16),DenseLayer(3,activation_function=None)],as_placeholders=False)
f4 = dnn4.forward()

output = f4[-1]

loss = dnn4.squared_cost()
tf.summary.scalar('loss', loss)
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(dnn.y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# Merge all the summaries and write them out to /tmp/mnist_logs (by default)
merged = tf.summary.merge_all()
#to save model
saver = tf.train.Saver()

with tf.Session() as sess:
    #initialize variables
    sess.run(tf.global_variables_initializer())

    #write training data to file in order to visualize it with tensorboard
    log_name = datetime.now().strftime("%Y%m%d-%H%M%S")
    #train_writer = tf.summary.FileWriter('summaries/train/logs'+log_name,sess.graph)
    test_writer = tf.summary.FileWriter('summaries/test/logs'+log_name,sess.graph)
    
    #training 
    for i in tqdm(range(epochs)):
        #mini batch training with my super generator function
        for c,batch in enumerate(get_batches(X_train,y_train,batch_size)):
            batch_x = batch[0]
            batch_y = batch[1]   
            feed_dict={dnn.X : batch_x,dnn.y : batch_y}
            cost,_,acc= sess.run([loss,train,accuracy],feed_dict=feed_dict)
            #print("cost in "+str(c)+' batch : '+str(cost))
            #print("accuracy in "+str(c)+' batch : '+str(acc))
        
        #test loggingfor every epoch
        feed_dict={dnn.X:X_test, dnn.y:y_test}
        summary,test_acc,test_cost = sess.run([merged,accuracy,loss],feed_dict=feed_dict)
        test_writer.add_summary(summary, i)
        
#         if i%10==0:
#             print('epoch:'+str(i))
            #print("cost after "+str(i)+' epoch : '+str(train_cost))
            #print("accuracy after "+str(i)+' epoch : '+str(train_acc))
           
    #save model (can save it if its better that the previous one etc)
    saver.save(sess,'models/mymodel.ckpt')
        
    feed_dict={dnn.X : X_test,dnn.y : y_test}
    cost,test_acc = sess.run([loss,accuracy],feed_dict=feed_dict)
    print("test accuracy "+str(test_acc))


  6%|████▉                                                                              | 3/50 [00:44<12:03, 15.40s/it]

KeyboardInterrupt: 